In [1]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [3]:
os.makedirs("images",exist_ok=True)
os.makedirs("data/mnist",exist_ok=True)

In [4]:
cuda = True if torch.cuda.is_available() else False

In [5]:
cuda

False

In [7]:
epochs = 200
batch_size = 64
lr = 0.0005
n_cpu = 8
latent_dim = 100
img_size = 28 #MNIST的大小为28x28
channels = 1
n_critic = 5 #训练discriminator次数
clip_value = 0.01 #梯度裁剪
sample_interval = 400 #生成图片保存间隔次数

In [9]:
img_shape = (channels,img_size,img_size) #图片大小 1x28x28

In [8]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist/",
        train = True,
        download = True,
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])]),
        #transforms.Normalize([0.5],[0.5])表示对单通道0-1图像进行归一化
    ),
    batch_size = batch_size,
    shuffle = True,
)

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [11]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity



In [12]:
generator = Generator()
discriminator = Discriminator()

In [13]:
optimizer_G = torch.optim.RMSprop(generator.parameters(),lr=lr)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(),lr=lr)

In [14]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [19]:
batches_done = 0
for epoch in range(epochs):

    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

        # Train the generator every n_critic iterations
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))

            loss_G.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )

        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        batches_done += 1


[Epoch 0/200] [Batch 0/938] [D loss: -2.240975] [G loss: -2.978066]
[Epoch 0/200] [Batch 5/938] [D loss: -4.061914] [G loss: -6.614383]
[Epoch 0/200] [Batch 10/938] [D loss: -4.037419] [G loss: -4.930456]
[Epoch 0/200] [Batch 15/938] [D loss: -2.029236] [G loss: -9.184769]
[Epoch 0/200] [Batch 20/938] [D loss: -0.216377] [G loss: -11.788095]
[Epoch 0/200] [Batch 25/938] [D loss: -0.698784] [G loss: -11.526352]
[Epoch 0/200] [Batch 30/938] [D loss: -1.013708] [G loss: -12.333344]
[Epoch 0/200] [Batch 35/938] [D loss: -0.455487] [G loss: -11.811468]
[Epoch 0/200] [Batch 40/938] [D loss: -0.312089] [G loss: -8.092979]
[Epoch 0/200] [Batch 45/938] [D loss: -0.634043] [G loss: -6.626289]
[Epoch 0/200] [Batch 50/938] [D loss: -0.525067] [G loss: -7.674510]
[Epoch 0/200] [Batch 55/938] [D loss: -0.528797] [G loss: -9.533457]
[Epoch 0/200] [Batch 60/938] [D loss: -0.356720] [G loss: -11.309855]
[Epoch 0/200] [Batch 65/938] [D loss: -0.538991] [G loss: -12.011827]
[Epoch 0/200] [Batch 70/938] [

[Epoch 0/200] [Batch 590/938] [D loss: 0.026704] [G loss: -0.868220]
[Epoch 0/200] [Batch 595/938] [D loss: -0.013038] [G loss: -1.315100]
[Epoch 0/200] [Batch 600/938] [D loss: -0.007877] [G loss: -2.039927]
[Epoch 0/200] [Batch 605/938] [D loss: -0.009734] [G loss: -2.358073]
[Epoch 0/200] [Batch 610/938] [D loss: -0.009181] [G loss: -2.097827]
[Epoch 0/200] [Batch 615/938] [D loss: 0.019435] [G loss: -0.865177]
[Epoch 0/200] [Batch 620/938] [D loss: -0.000569] [G loss: -0.520749]
[Epoch 0/200] [Batch 625/938] [D loss: -0.018529] [G loss: -0.447390]
[Epoch 0/200] [Batch 630/938] [D loss: -0.013239] [G loss: -0.426689]
[Epoch 0/200] [Batch 635/938] [D loss: -0.010958] [G loss: -0.454448]
[Epoch 0/200] [Batch 640/938] [D loss: -0.006824] [G loss: -0.534076]
[Epoch 0/200] [Batch 645/938] [D loss: 0.009342] [G loss: -0.777457]
[Epoch 0/200] [Batch 650/938] [D loss: 0.004092] [G loss: -0.999672]
[Epoch 0/200] [Batch 655/938] [D loss: -0.009307] [G loss: -1.643159]
[Epoch 0/200] [Batch 660

[Epoch 1/200] [Batch 245/938] [D loss: -1.039532] [G loss: -2.385560]
[Epoch 1/200] [Batch 250/938] [D loss: -0.209688] [G loss: -2.174947]
[Epoch 1/200] [Batch 255/938] [D loss: -0.548911] [G loss: -0.917580]
[Epoch 1/200] [Batch 260/938] [D loss: -0.588879] [G loss: -1.098673]
[Epoch 1/200] [Batch 265/938] [D loss: -0.242930] [G loss: -1.527581]
[Epoch 1/200] [Batch 270/938] [D loss: -0.070303] [G loss: -1.042258]
[Epoch 1/200] [Batch 275/938] [D loss: 0.065943] [G loss: -2.584378]
[Epoch 1/200] [Batch 280/938] [D loss: 0.154636] [G loss: -1.667094]
[Epoch 1/200] [Batch 285/938] [D loss: 0.047676] [G loss: -0.753178]
[Epoch 1/200] [Batch 290/938] [D loss: -0.122774] [G loss: -0.153533]
[Epoch 1/200] [Batch 295/938] [D loss: -0.393363] [G loss: 1.162203]
[Epoch 1/200] [Batch 300/938] [D loss: -0.015100] [G loss: -0.463344]
[Epoch 1/200] [Batch 305/938] [D loss: -0.025173] [G loss: -0.634477]
[Epoch 1/200] [Batch 310/938] [D loss: -0.032258] [G loss: -0.601668]
[Epoch 1/200] [Batch 315

[Epoch 1/200] [Batch 835/938] [D loss: -0.200225] [G loss: -1.322426]
[Epoch 1/200] [Batch 840/938] [D loss: -0.305609] [G loss: -1.348875]
[Epoch 1/200] [Batch 845/938] [D loss: -0.150579] [G loss: -0.909575]
[Epoch 1/200] [Batch 850/938] [D loss: -0.246851] [G loss: -0.785280]
[Epoch 1/200] [Batch 855/938] [D loss: -0.229586] [G loss: -1.252811]
[Epoch 1/200] [Batch 860/938] [D loss: -0.171058] [G loss: -1.394660]
[Epoch 1/200] [Batch 865/938] [D loss: -0.268281] [G loss: -1.098289]
[Epoch 1/200] [Batch 870/938] [D loss: -0.298596] [G loss: -0.498740]
[Epoch 1/200] [Batch 875/938] [D loss: -0.195195] [G loss: -0.036120]
[Epoch 1/200] [Batch 880/938] [D loss: -0.237895] [G loss: -0.621758]
[Epoch 1/200] [Batch 885/938] [D loss: -0.308880] [G loss: -1.000762]
[Epoch 1/200] [Batch 890/938] [D loss: -0.270429] [G loss: -1.075319]
[Epoch 1/200] [Batch 895/938] [D loss: -0.281636] [G loss: -0.652178]
[Epoch 1/200] [Batch 900/938] [D loss: -0.343721] [G loss: -0.043092]
[Epoch 1/200] [Batch

[Epoch 2/200] [Batch 490/938] [D loss: -0.111259] [G loss: 0.830574]
[Epoch 2/200] [Batch 495/938] [D loss: -0.216888] [G loss: 0.426347]
[Epoch 2/200] [Batch 500/938] [D loss: -0.242157] [G loss: -0.702350]
[Epoch 2/200] [Batch 505/938] [D loss: -0.167673] [G loss: -1.580973]
[Epoch 2/200] [Batch 510/938] [D loss: -0.157091] [G loss: -1.224915]
[Epoch 2/200] [Batch 515/938] [D loss: -0.112746] [G loss: -0.441239]
[Epoch 2/200] [Batch 520/938] [D loss: -0.168298] [G loss: -0.500459]
[Epoch 2/200] [Batch 525/938] [D loss: -0.226759] [G loss: -0.495269]
[Epoch 2/200] [Batch 530/938] [D loss: -0.244283] [G loss: -0.429689]
[Epoch 2/200] [Batch 535/938] [D loss: -0.195449] [G loss: -1.298737]
[Epoch 2/200] [Batch 540/938] [D loss: -0.100311] [G loss: -0.889207]
[Epoch 2/200] [Batch 545/938] [D loss: -0.227618] [G loss: -0.455988]
[Epoch 2/200] [Batch 550/938] [D loss: -0.226823] [G loss: 0.036489]
[Epoch 2/200] [Batch 555/938] [D loss: -0.139252] [G loss: 0.231773]
[Epoch 2/200] [Batch 560

...

[Epoch 197/200] [Batch 10/938] [D loss: -0.013615] [G loss: 0.212818]
[Epoch 197/200] [Batch 15/938] [D loss: 0.010112] [G loss: 0.012775]
[Epoch 197/200] [Batch 20/938] [D loss: 0.000612] [G loss: 0.399280]
[Epoch 197/200] [Batch 25/938] [D loss: -0.003113] [G loss: 0.187924]
[Epoch 197/200] [Batch 30/938] [D loss: -0.001647] [G loss: -0.339741]
[Epoch 197/200] [Batch 35/938] [D loss: -0.015182] [G loss: 0.255847]
[Epoch 197/200] [Batch 40/938] [D loss: -0.008907] [G loss: -0.484903]
[Epoch 197/200] [Batch 45/938] [D loss: -0.002641] [G loss: -0.092484]
[Epoch 197/200] [Batch 50/938] [D loss: -0.008758] [G loss: 0.432278]
[Epoch 197/200] [Batch 55/938] [D loss: -0.019513] [G loss: -0.076154]
[Epoch 197/200] [Batch 60/938] [D loss: -0.008975] [G loss: -0.304922]
[Epoch 197/200] [Batch 65/938] [D loss: -0.014458] [G loss: 0.253287]
[Epoch 197/200] [Batch 70/938] [D loss: -0.002265] [G loss: -0.298913]
[Epoch 197/200] [Batch 75/938] [D loss: 0.002972] [G loss: -0.084391]
[Epoch 197/200] 

[Epoch 197/200] [Batch 590/938] [D loss: -0.011467] [G loss: 0.480145]
[Epoch 197/200] [Batch 595/938] [D loss: -0.002549] [G loss: 0.369101]
[Epoch 197/200] [Batch 600/938] [D loss: -0.004130] [G loss: 0.570935]
[Epoch 197/200] [Batch 605/938] [D loss: 0.010791] [G loss: 0.569244]
[Epoch 197/200] [Batch 610/938] [D loss: -0.002183] [G loss: 0.707239]
[Epoch 197/200] [Batch 615/938] [D loss: -0.005342] [G loss: 0.370701]
[Epoch 197/200] [Batch 620/938] [D loss: 0.004934] [G loss: -0.189571]
[Epoch 197/200] [Batch 625/938] [D loss: -0.014133] [G loss: -0.396780]
[Epoch 197/200] [Batch 630/938] [D loss: -0.044342] [G loss: -0.672762]
[Epoch 197/200] [Batch 635/938] [D loss: 0.019441] [G loss: -0.312946]
[Epoch 197/200] [Batch 640/938] [D loss: -0.003801] [G loss: -0.159964]
[Epoch 197/200] [Batch 645/938] [D loss: 0.006461] [G loss: 0.018654]
[Epoch 197/200] [Batch 650/938] [D loss: -0.001508] [G loss: -0.001605]
[Epoch 197/200] [Batch 655/938] [D loss: -0.003303] [G loss: 0.189056]
[Epo

[Epoch 198/200] [Batch 235/938] [D loss: -0.006467] [G loss: 0.142817]
[Epoch 198/200] [Batch 240/938] [D loss: -0.014380] [G loss: 0.440535]
[Epoch 198/200] [Batch 245/938] [D loss: -0.030340] [G loss: 0.682716]
[Epoch 198/200] [Batch 250/938] [D loss: -0.020484] [G loss: 0.803493]
[Epoch 198/200] [Batch 255/938] [D loss: -0.011231] [G loss: 0.695679]
[Epoch 198/200] [Batch 260/938] [D loss: 0.000799] [G loss: -0.009099]
[Epoch 198/200] [Batch 265/938] [D loss: -0.029221] [G loss: -0.091049]
[Epoch 198/200] [Batch 270/938] [D loss: -0.033625] [G loss: -0.980389]
[Epoch 198/200] [Batch 275/938] [D loss: -0.003574] [G loss: -0.902664]
[Epoch 198/200] [Batch 280/938] [D loss: 0.002643] [G loss: -0.598507]
[Epoch 198/200] [Batch 285/938] [D loss: -0.014088] [G loss: -0.424084]
[Epoch 198/200] [Batch 290/938] [D loss: -0.020236] [G loss: -0.681323]
[Epoch 198/200] [Batch 295/938] [D loss: -0.021627] [G loss: -0.235961]
[Epoch 198/200] [Batch 300/938] [D loss: -0.003914] [G loss: 0.245864]


[Epoch 198/200] [Batch 815/938] [D loss: -0.017092] [G loss: 0.045237]
[Epoch 198/200] [Batch 820/938] [D loss: -0.055200] [G loss: -0.569807]
[Epoch 198/200] [Batch 825/938] [D loss: -0.022714] [G loss: -0.963289]
[Epoch 198/200] [Batch 830/938] [D loss: 0.018648] [G loss: -0.346899]
[Epoch 198/200] [Batch 835/938] [D loss: -0.005983] [G loss: -0.099668]
[Epoch 198/200] [Batch 840/938] [D loss: -0.001919] [G loss: 0.385938]
[Epoch 198/200] [Batch 845/938] [D loss: -0.007729] [G loss: 0.548516]
[Epoch 198/200] [Batch 850/938] [D loss: 0.003161] [G loss: 0.434062]
[Epoch 198/200] [Batch 855/938] [D loss: 0.015074] [G loss: 0.331114]
[Epoch 198/200] [Batch 860/938] [D loss: 0.000547] [G loss: -0.075643]
[Epoch 198/200] [Batch 865/938] [D loss: 0.018188] [G loss: -0.191401]
[Epoch 198/200] [Batch 870/938] [D loss: -0.007086] [G loss: -0.226173]
[Epoch 198/200] [Batch 875/938] [D loss: -0.002175] [G loss: 0.162972]
[Epoch 198/200] [Batch 880/938] [D loss: 0.003587] [G loss: 0.163965]
[Epoc

[Epoch 199/200] [Batch 455/938] [D loss: 0.010667] [G loss: 0.645767]
[Epoch 199/200] [Batch 460/938] [D loss: -0.009413] [G loss: -0.042152]
[Epoch 199/200] [Batch 465/938] [D loss: -0.016632] [G loss: -0.025347]
[Epoch 199/200] [Batch 470/938] [D loss: 0.013139] [G loss: -0.152120]
[Epoch 199/200] [Batch 475/938] [D loss: -0.008819] [G loss: -0.364632]
[Epoch 199/200] [Batch 480/938] [D loss: 0.028477] [G loss: -0.048568]
[Epoch 199/200] [Batch 485/938] [D loss: -0.005084] [G loss: -0.028686]
[Epoch 199/200] [Batch 490/938] [D loss: -0.013663] [G loss: -0.149795]
[Epoch 199/200] [Batch 495/938] [D loss: -0.003707] [G loss: -0.231772]
[Epoch 199/200] [Batch 500/938] [D loss: -0.017171] [G loss: 0.375469]
[Epoch 199/200] [Batch 505/938] [D loss: -0.015654] [G loss: 0.358437]
[Epoch 199/200] [Batch 510/938] [D loss: -0.001974] [G loss: 0.238873]
[Epoch 199/200] [Batch 515/938] [D loss: -0.022131] [G loss: 0.123341]
[Epoch 199/200] [Batch 520/938] [D loss: -0.027320] [G loss: -0.624644]
